In [ ]:
%config Completer.use_jedi=False

import sys
sys.path.append('../../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dementmutant

from refsimu import prepare_simu_from_seeds


def display_metabolite_params(eco, metname):
    print('{}\n Metabolic coefficients for {}\n{}'.format("*"*50,metname,"*"*50))
    met = eco.microbes.metabolites[metname]
    display(met.get_map_float_df())
    for k in ['Consti','Induci']:
        print('{}\n{}\n{}'.format('*'*50,k,'*'*50))
        print('Weights {}'.format(metname))
        display(met.get_weight_map_df(k))
        display(met.get_weight_map_df(k).sum(axis=1).mean())
        #
        print('Per taxon global factor {} {}'.format(k,metname))
        display(met.get_ref_cos_v_df(k))
        print("C based effective costs {} {}".format(k,metname))
        display(met.get_ref_cost_df(k))
        tt = met.get_ref_cost_df(k)
        print('Ref cost per taxon sum {} {}'.format(k,metname))
        ss = pd.DataFrame(tt.sum(axis=1))
        display(ss)
        print('Ref cost per taxon sum stats')
        display(ss.describe())
        print("Metabolite production stoechiometry multiplicators -{} {}".format(k,metname))
        display(met.rel_cost[k])

        
def get_upt_efficiency_cellulose(eco):
    env = eco.functions_module.env_func_constant(0)
    csts_induci = (
            eco.microbes.get_metabolic_processes_operands('Induci',env)
            )
    csts_induci_C = (1.0-csts_induci[1][0,:,0])*(1-csts_induci[1][1,:,0])

    h0 = csts_induci_C[0]
    h1 = csts_induci_C[-1]
    xc = 0.5
    gc = 1.0
    hc = h0 + xc*(h1-h0)
    g0 = gc * (1.0+xc*(h1-h0) / hc)
    g1 = g0 - gc *(h1-h0) / hc
    return g0,g1,h0,h1

def get_ops(eco,process='Consti'):
    env = eco.functions_module.env_func_constant(0)
    csts = (
            eco.microbes.get_metabolic_processes_operands(process,env)
            )
    return csts

def compute_maxCuptake_permon_full(eco,imon, mon_sat_C = 10000.0,itime=0, with_induci=False):
    eco.monomers.mass[()] = 0
    eco.monomers.mass[:,:,imon,:] = mon_sat_C
    eco.monomers.update_biomass_fields()
#     eco.microbes.mass[:,:,:,0] = 1.0
    eco.microbes.mass[:,:,:,:] = 1.0
    eco.microbes.update_biomass_fields()
    eco.apply_monomer_uptake_only(itime)
#     print('imon = {}'.format(imon))
#     print(eco.microbes.delta_mass[0,0,:,0])
    if (with_induci):
        eco.apply_inducible_only(itime)
    else:
#         print(eco.microbes.mass[0,0,:,0])
        eco.apply_maxuptake_to_microbes(itime)
#         print(eco.microbes.mass[0,0,:,0])
    return eco
    

def compute_maxCuptake_allmons_full(eco,mon_sat_C=10000.0,itime=0, with_induci=False):
    tmp_l = []
    for imon in range(eco.monomers.n_components):
        eco = compute_maxCuptake_permon_full(eco,imon,mon_sat_C,itime,with_induci)
        tmp_l.append(eco.microbes.mass[0,0,:,0].copy()-1.0)
    res = np.column_stack(tmp_l)
    df = pd.DataFrame(data=res, index = eco.microbes.names, columns=eco.monomers.names,copy=True)
    return res,df


def compute_constitutive_rate_full(eco,itime=0):
    eco.microbes.mass[()]  = 0.0
    eco.microbes.mass[:,:,:,0]  = 1.0
    eco.microbes.mass[:,:,:,1]  = 1.0
    eco.microbes.update_biomass_fields()
    eco.microbes.delta_mass[()] = 0.0
    eco.apply_metabolic_processes('Consti',itime)
    res = 1.0-eco.microbes.mass[0,0,:,0].copy()
    df = pd.DataFrame(data=res,index=eco.microbes.names,columns=['C'],copy=True)
    return res,df


def get_meta_cost(eco,process='Consti'):
    env = eco.get_environment(0)
    t = eco.microbes.get_metabolic_processes_operands(process,env)
    return t

In [ ]:
grid_shape = (1,1)
n_degrad_enzymes = 12
n_taxa = 100
n_osmolytes = 1

case_parameters = {
               'substrate_inputs' :  {'name':'TESTINPPUT','steps':[('DementDefault','inf')]},
               'initial_diversity':'uniform',
               'mutation_rate':None,
               'disp_proba0': 0.8,
               }

In [ ]:
eco = prepare_simu_from_seeds(grid_shape,
                              n_degrad_enzymes,
                              n_taxa,
                              n_osmolytes,
                              case_parameters=case_parameters
                             )

In [ ]:
csts_induci = get_ops(eco,'Induci')

In [ ]:
# dvals = [1.0e-3,1.5e-3,1.75e-3,2.0e-3]
# dvals = np.linspace(1.743e-3,1.7445e-3,10)
# # dvals = np.linspace(0.001,0.01,100)
# dvals = np.linspace(1e-4,1e-3,64)
# eff_upt_vals = []
# consti_df_vals = []
# for i,delta_eff in enumerate(dvals):
#     eco = prepare_simu_from_seeds(grid_shape,
#                               n_degrad_enzymes,
#                               n_taxa,
#                               n_osmolytes,
#                               delta_eff=delta_eff
#                              )
#     eff_upt, eff_upt_df = compute_maxCuprake_allmons_full(eco,with_induci=True)
#     eff_upt_vals.append(eff_upt_df)
#     consti_r, consti_df = compute_constitutive_rate_full(eco)
#     consti_df_vals.append(consti_df)
# fix, ax = plt.subplots(1,1,figsize=(8,8))
# xplt = np.array(range(eco.microbes.n_taxa))
# for delta_eff,eff_df,cons_df in zip(dvals, eff_upt_vals,consti_df_vals):
#     ax.plot(xplt,eff_df['Cellulose'],label='{}'.format(delta_eff))
# # ax.legend() ;


# fix, ax = plt.subplots(1,1,figsize=(8,8))
# xplt = np.array(range(eco.microbes.n_taxa))
# for delta_eff,eff_df,cons_df in zip(dvals, eff_upt_vals,consti_df_vals):
#     ax.plot(xplt,cons_df['C'],label='{}'.format(delta_eff))
# # ax.legend() ;

# fix, ax = plt.subplots(1,1,figsize=(8,8))
# xplt = np.array(range(eco.microbes.n_taxa))
# for delta_eff,eff_df,cons_df in zip(dvals, eff_upt_vals,consti_df_vals):
#     ax.plot(xplt,cons_df['C'] /eff_df['Cellulose'],label='{}'.format(delta_eff))
# # ax.legend() ;

In [ ]:
# eco.uptake_transporters.get_base_kinetic_parameter_df('Vmax')

In [ ]:
met = eco.microbes.metabolites['Degradation_Enzymes']

In [ ]:
met.get_metabolite_index('DegEnz3')

In [ ]:
met.ref_cost['Induci']

In [ ]:
eco.monomer_uptake_op.get_taxa_relative_efficiency_df()

In [ ]:
eco.degradation_enzymes.get_map_bool_df()

In [ ]:
print(eco.microbes.metabolites.keys())

### Degradation enzymes production efficiency

In [ ]:
display_metabolite_params(eco,'Degradation_Enzymes')

### Uptake transporters production costs

In [ ]:
display_metabolite_params(eco,'Uptake_Transporters')

In [ ]:
#display_metabolite_params(eco,'Respiration_Growth')

In [ ]:
#display_metabolite_params(eco,'Osmolytes')

In [ ]:
# %matplotlib inline
# for itax,tax in enumerate(eco.microbes.names):
#     dat = eco.microbes.tmass[:,:,itax,0]
#     fig,ax = plt.subplots(1,1)
#     im_mass = ax.imshow(dat,cmap='hot')
#     ax.set_title('total mass {}'.format(tax))
#     plt.colorbar(im_mass,ax=ax)
    

In [ ]:
max_upt, max_upt_df = compute_maxCuptake_allmons_full(eco)

In [ ]:
max_upt_df

In [ ]:
k2 = (max_upt_df['Cellulose'][-1]-max_upt_df['Cellulose'][0]) / (1.8)
k1 = max_upt_df['Cellulose'][-1]-k2 * 2
k1b = max_upt_df['Cellulose'][0]- k2 * 0.2
print(k2)
print(k1)
print(k1b)

In [ ]:
eff_upt, eff_upt_df = compute_maxCuptake_allmons_full(eco,with_induci=True)

In [ ]:
eff_upt_df

In [ ]:
truc = eff_upt_df / max_upt_df
display(truc)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,4))
xplt = np.array(range(eco.microbes.n_taxa))
ax.plot(xplt,eff_upt_df['Cellulose'])
# ax.plot(xplt,max_upt_df['Cellulose'])
ax.grid();

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,4))
xplt = np.array(range(eco.microbes.n_taxa))
ax.plot(xplt,max_upt_df['Cellulose'])
ax.grid();

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,4))
xplt = np.array(range(eco.microbes.n_taxa))
ax.plot(xplt,eff_upt_df['Cellulose']/max_upt_df['Cellulose'])
# ax.plot(xplt,max_upt_df['Cellulose'])
ax.grid();

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(14,4))
xplt = np.array(range(eco.microbes.n_taxa))
for cc in eff_upt_df.columns:
#     ax.plot(xplt,eff_upt_df[cc],label=cc)
    axes[0].plot(xplt,max_upt_df[cc],label=cc)
    if (cc == 'Cellulose'):
        continue
    axes[1].plot(xplt,max_upt_df[cc],label=cc)
for ax in axes:        
    ax.grid();
    ax.set_xlabel("taxa")
    ax.legend();
    ax.set_title('Max uptake ')

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(14,4))
xplt = np.array(range(eco.microbes.n_taxa))
for cc in eff_upt_df.columns:
    axes[0].plot(xplt,eff_upt_df[cc],label=cc)
    if (cc == 'Cellulose'):
        continue
    axes[1].plot(xplt,eff_upt_df[cc],label=cc)
for ax in axes:        
    ax.grid();
    ax.set_xlabel("taxa")
    ax.legend();
    ax.set_title('Effective max uptake (with induci)')

In [ ]:
consti,consti_df = compute_constitutive_rate_full(eco)

In [ ]:
consti_df

In [ ]:
s, g0,g1 = case_parameters['uptake_c_scale'],case_parameters['uptake_cellulose_c0'],case_parameters['uptake_cellulose_c1']

# total constitutive loss rate as a function of trait and parameters
truc1 = 0.007203 + s*(0.13 + 0.01 * (g0 +(g1-g0)* np.linspace(0,1,n_taxa)))
print(truc1)

# max uptake for non cellulose
truc2 = 2.680123 * s
print(truc2)
# truc2 = 0.268012
# max uptake for cellulose
# truc3 = 0.053602 + (0.536025-0.053602) * np.linspace(0,1,n_taxa)
# print(truc3)
k2 = 2.6801234381934838
k1 = 0
truc3 = s*(k1 + k2 * (g0 +(g1-g0)* np.linspace(0,1,n_taxa)))
print(truc3)


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,4))
xplt = np.array(range(eco.microbes.n_taxa))
ax.plot(xplt,consti_df['C']/eff_upt_df['Cellulose'])
ax.plot(xplt,np.ones_like(xplt))
# ax.plot(xplt,max_upt_df['Cellulose'])
ax.grid();

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(14,4))
xplt = np.array(range(eco.microbes.n_taxa))
for cc in eff_upt_df.columns:
    axes[0].plot(xplt,consti_df['C']/eff_upt_df[cc],label=cc)
    if (cc == 'Cellulose'):
        continue
    axes[1].plot(xplt,consti_df['C']/eff_upt_df[cc],label=cc)
for ax in axes:
    ax.grid();
    ax.legend();
    ax.set_title(r'$\rho_{critical}$')

In [ ]:
# #
# source_key = list(eco.source_operators.keys())[0]
# print(source_key)
# source_op = eco.source_operators[source_key]
# sub_source = source_op.source_func(eco.substrates,0)[0,0,:,:]
# print(sub_source.shape)
# sub_source_df = pd.DataFrame(data=sub_source,index=eco.substrates.names,columns=['C','N','P'])
# # display(sub_source_df)
# sub_source_df['T'] = sub_source_df.sum(axis=1)
# display(sub_source_df)
# display(pd.DataFrame(sub_source_df.sum(axis=0)))

# sub_source_rel_df = sub_source_df.divide(sub_source_df.sum(axis=0))
# display(sub_source_rel_df)
# # display(sub_source_rel_df.sum(axis=0))
# # sub_source_tmass1 = np.sum(sub_source,axis=0)
# # source_stoech = sub_source_tmass1 / np.sum(sub_source_tmass1)
# # print(sub_source)
# # print(sub_source_tmass1)
# # print(source_stoech)
# # print(np.sum(source_stoech))
# # sub_source_tmass1_mod = sub_source_tmass1 * np.array([0.4,1,1])
# # source_stoech_mod = sub_source_tmass1_mod / np.sum(sub_source_tmass1_mod)
# # print(sub_source_tmass1_mod)
# # print(source_stoech_mod)

# #
# sub_source_tmass2 = np.sum(sub_source,axis=1,)
# print(sub_source_tmass2)
# sub_source_relw_tmass = sub_source_tmass2 / np.sum(sub_source_tmass2)
# print(sub_source_relw_tmass)

In [ ]:
# eco.microbes.stoechio_balance_recycler.stoechiometry_quotas

In [ ]:
def get_sample_uptake(eff_df,rho_cell,rho_ncell):
    subs = eff_upt_df.columns
    nsubs = eff_df.columns.size
    rho_vals = pd.Series(index=eff_upt_df.columns,data = rho_ncell * np.ones((nsubs,)))
    rho_vals['Cellulose'] = rho_cell
    eff_upt = eff_df @ rho_vals
#     display(eff_upt)
    return eff_upt


def scan_sample_uptake(eff_df, const_df):
    N1 = 32
    N2 = 16
    rho_cell_vals = np.linspace(0,1,N1)
    rho_ncell_vals = np.linspace(0,1,N2)
    RC,RNC = np.meshgrid(rho_cell_vals,rho_ncell_vals, indexing='ij')
    ntax = eff_df.index.size
#     res = {k: np.zeros_like(RC) for k in range(ntax)}
    res = np.zeros((ntax,RC.shape[0],RC.shape[1]))
    I1,I2 = np.indices(RC.shape)
    for i1,i2 in zip(I1.ravel(),I2.ravel()):
        tmp_r = get_sample_uptake(eff_df,RC[i1,i2],RNC[i1,i2])
        for k in range(ntax):
            res[k,i1,i2] = tmp_r[k]-const_df[k]
    return res,RC,RNC
    
    

In [ ]:
samp_up,RC,RNC = scan_sample_uptake(eff_upt_df,consti_df['C'])

In [ ]:
itax = 5
fig,ax = plt.subplots(1,1,figsize=(3,3))
tmp_im = ax.contourf(RC,RNC,samp_up[itax,:,:],levels=50,cmap='hot')
plt.colorbar(tmp_im,ax=ax)
#     ax.axis('off')
ax.set_title('Tax {}'.format(itax))
ax.set_xlabel(r'$\rho_{cellullose}$')
ax.set_ylabel(r'$\rho_{other}$');

In [ ]:
for incell in [0,1,2,8,12]:
    fig,ax = plt.subplots(1,1,figsize=(12,6))
    rho_ncell = RNC[0,incell]
    xr = np.array(range(samp_up.shape[0]))
    for icell,rho_cell in enumerate(RC[:,incell]): 
        ax.plot(xr,samp_up[:,icell,incell],label='{:.2f}'.format(rho_cell))
    ax.grid()
    ax.legend(fontsize=6,ncol=4)
    ax.set_title('rho ncell {:.2f}'.format(rho_ncell))
    ax.set_xlabel('Taxa');
    ax.set_ylabel('Growth rate estimation');
    plt.show()
    plt.close(fig)